In [117]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

#### Functions for importing, cleaning, and sub-setting data XRF whole rock chemistry data.

In [118]:
## CSV to pandas DataFrame Import##
from tabulate import tabulate
def csv_to_df(filename, filepath=None):
    """
    Extracts major and trace/REE element oxide data from a .csv and organizes it into a pandas DataFrame.
            - Each column is a different oxide/species/category.
            - Each sample is a row in the .csv file.
            - Each sample becomes a row in a pandas DataFrame.
            - Data can be filtered by sample name, oxide name, or special groups:
                - Major elements
                - Trace elements
                - REE elements

    ****At the moment, all data includes the LOI + sum values.

    Parameters:
        - filename: str - name of the .csv file
        - **folderpath: str, optional: path to folder if not sci-data

    Returns:
        pd.DataFrame: The imported data as a pandas DataFrame.
    """
    if filepath is None:
        script_dir = os.path.expanduser('~/PycharmProjects/scientific-coding-v1/sci-data/')
        filepath = os.path.join(script_dir, filename)

        df = pd.read_csv(filepath)
        column_names = list(df.columns)

        print(f"Columns:{column_names}")
        print(f"\n .csv file successfully imported.")
    return df

def extract_MajorOx(df):
    """
    Extracts major element oxides from a pandas DataFrame.

    Places them in a list alongside oxides expected by MELTS as input.

        MAJOR OXIDES:
            - SiO2
            - TiO2
            - Al2O3
            - Fe2O3
            - Cr2O3
            - FeO
            - MnO
            - MgO
            - NiO
            - CoO
            - CaO
            - Na2O
            - K2O
            - P2O5

        VOLATILE PHASES:
            - H2O
            - CO2
            - SO3
            - Cl2O-1
            - F2O-1
    """
    major_oxides = ['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'Cr2O3', 'FeO', 'MnO', 'MgO', 'NiO', 'CoO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'H2O', 'CO2', 'SO3', 'Cl2O-1', 'F2O-1']
    major_df = df[major_oxides]
    print("\n Major oxides extracted.")
    return major_df

## Test

In [130]:
rawdata = csv_to_df('tuolumnedata_csvready.csv')

# Use function to extract major oxides, fill NaN values with '0', ensure all values are float types for future calculations.
cleaned_df = extract_MajorOx(rawdata)
cleaned_df = cleaned_df.fillna(0)
cleaned_df = cleaned_df.replace('n.d.',0)
cleaned_df = cleaned_df.astype(float)

Columns:['Sample', 'Type', 'SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'Cr2O3', 'FeO', 'MnO', 'MgO', 'NiO', 'CoO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'H2O', 'CO2', 'SO3', 'Cl2O-1', 'F2O-1', 'LOI (%)', 'sumMaj+LOI', 'sumAll', 'F >=', 'Cl >=', 'SO3 >=', 'Br >=', 'As >=', 'Ni', 'Cr', 'V', 'Sc', 'Cu', 'Zn', 'Ga', 'Ba', 'Rb', 'Cs', 'Sr', 'Y', 'Zr', 'Hf', 'Nb', 'Ta', 'Mo', 'La', 'Ce', 'Nd', 'Sm', 'Dy', 'Yb', 'Th', 'U', 'Tl', 'Pb', 'Sn', 'Bi', 'Sb']

 .csv file successfully imported.

 Major oxides extracted.


### Error calculations for experimental XRF data

In [120]:
## Should be eventually replaced with a scaleable loop, but for now, this works.
error_table = rawdata.iloc[-6:, 2:]
error_table = error_table.fillna(0)
error_table = error_table.replace('n.d.',0)

error_table=error_table.astype(float)
error1 = abs(error_table.iloc[0,:] - error_table.iloc[3,:]) #Absolute value of experimental error
error_table.loc['error1'] = error1

error2 = abs(error_table.iloc[1,:] - error_table.iloc[4,:]) #Absolute value of experimental error
error_table.loc['error2'] = error2

error3 = abs(error_table.iloc[2,:] - error_table.iloc[5,:]) #Absolute value of experimental error
error_table.loc['error3'] = error3

avg_error = (error1 + error2 + error3) / 3
error_table.loc['avg_error'] = avg_error

cleaned_df.loc['XRF_error'] = avg_error

error_table = error_table.iloc[-4:,:]

In [121]:
## Standard deviation
N=3
for i in range(N):
    s = (error_table.iloc[(i),:] - error_table.iloc[3,:])**2
    s = s * (1/3)
    s = np.sqrt(s)

In [138]:
for i in range(N):
    seom = s / np.sqrt(N)
error_table.loc['seom'] = seom
error_table = error_table.fillna(0)
error_table

,SiO2,TiO2,Al2O3,Fe2O3,Cr2O3,FeO,MnO,MgO,NiO,CoO,...,Sm,Dy,Yb,Th,U,Tl,Pb,Sn,Bi,Sb
error1,0.364497,0.003682,0.040144,0.0,0.0,0.110367,0.001144,0.019102,0.0,0.0,...,0.351393,0.497115,0.182302,0.137323,0.113917,0.761773,1.103507,2.985603,0.008599,3.641769
error2,0.624184,0.019661,0.037999,0.0,0.0,0.093277,0.003327,0.003784,0.0,0.0,...,0.190455,0.278685,1.348038,0.307694,0.293883,0.029954,0.201020,1.720000,0.113520,0.443106
error3,0.002948,0.002174,0.075641,0.0,0.0,0.072974,0.000455,0.004134,0.0,0.0,...,0.424768,0.074344,0.446952,0.395596,0.340000,0.047349,0.871338,0.880000,0.032311,0.220000
avg_error,0.330543,0.008506,0.051261,0.0,0.0,0.092206,0.001642,0.009007,0.0,0.0,...,0.322205,0.283381,0.659097,0.280205,0.249267,0.279692,0.725288,1.861868,0.051477,1.434958
seom,0.109198,0.002111,0.008127,0.0,0.0,0.006411,0.000396,0.001624,0.0,0.0,...,0.034187,0.069679,0.070715,0.038464,0.030244,0.077448,0.048683,0.327289,0.006389,0.404986


In [124]:
rawdata

,Sample,Type,SiO2,TiO2,Al2O3,Fe2O3,Cr2O3,FeO,MnO,MgO,...,Sm,Dy,Yb,Th,U,Tl,Pb,Sn,Bi,Sb
0,KHD105H,host,66.066406,0.548861,15.624385,NaN,NaN,3.732880,0.076576,1.524711,...,5.508537,2.776007,2.371975,18.246105,6.459493844,2.259647,15.638420,0.906214382,0.115245257,0.604142921
1,KKC103B,host,59.488558,0.885564,16.893312,NaN,NaN,6.088012,0.105994,3.120577,...,4.819822,3.500332,2.359592,13.146036,2.695031375,1.322610,12.617361,n.d.,0.096577265,0.511328718
2,KHD107B,host,63.811922,0.644781,16.617554,NaN,NaN,4.147709,0.085269,1.756226,...,4.195134,2.655695,2.431097,9.815123,3.943840476,0.945920,15.929009,0.621485015,0.137955528,0.621485015
3,KCP109D,host,68.591354,0.438112,15.640367,NaN,NaN,2.427771,0.063995,0.777234,...,3.353161,1.931738,2.061549,16.417172,7.082539115,1.703224,18.052295,n.d.,0.056440111,0.814113827
4,KCP109C,aplite,75.820463,0.081973,12.946199,NaN,NaN,0.663837,0.031223,0.089308,...,0.177724,0.768513,1.829517,27.457648,19.25596103,2.480307,29.240701,4.991387679,0.11624082,1.120515601
5,KCP114B,host,67.622928,0.475611,15.900443,NaN,NaN,2.794764,0.056634,0.875509,...,4.366113,2.150096,1.680311,5.363045,2.062842809,1.332152,15.753700,1.750695085,0.123902028,0.514910319
6,KCP109B,aplite,76.363613,0.069413,12.654557,NaN,NaN,0.517922,0.023925,0.054598,...,0.081400,0.669652,1.534719,25.257317,9.941924468,2.665886,30.857387,0.304664936,0.195434615,1.117104766
7,KHD106B,host,65.146071,0.546541,16.125992,NaN,NaN,3.654955,0.072742,1.580363,...,4.131962,2.907899,2.231251,33.060713,7.898540197,2.483403,16.001987,4.385579477,0.149595723,0.713931543
8,KCP109A,aplite,76.616213,0.079309,12.481429,NaN,NaN,0.623441,0.026245,0.055272,...,0.842885,0.664543,1.224120,35.431809,22.35686784,1.684341,32.008421,n.d.,0.141473315,1.107277575
9,KCP108-M,elizabeth,69.494990,0.447260,14.881052,NaN,NaN,2.559606,0.061803,0.804506,...,3.600613,1.960994,1.936984,12.260056,5.155742181,1.140728,17.728023,2.481114616,0.070342168,0.620278654


In [133]:
cleaned_df.loc['seom'] = seom

In [145]:
id = rawdata['Sample']
type = rawdata['Type']

0       KHD105H
1       KKC103B
2       KHD107B
3       KCP109D
4       KCP109C
5       KCP114B
6       KCP109B
7       KHD106B
8       KCP109A
9      KCP108-M
10      KCP110A
11    KHD107B-R
12      AGV-1-u
13        G-2-u
14       JA-1-u
15        AGV-1
16          G-2
17         JA-1
Name: Sample, dtype: object

In [129]:
cleaned_df

,SiO2,TiO2,Al2O3,Fe2O3,Cr2O3,FeO,MnO,MgO,NiO,CoO,...,Na2O,K2O,P2O5,H2O,CO2,SO3,Cl2O-1,F2O-1,type,sample
0,66.066406,0.548861,15.624385,0.0,0.0,3.732880,0.076576,1.524711,0.0,0.0,...,3.642991,3.377294,0.179945,0.0,0.0,0.0,0.0,0.0,host,KHD105H
1,59.488558,0.885564,16.893312,0.0,0.0,6.088012,0.105994,3.120577,0.0,0.0,...,3.472646,2.432116,0.214490,0.0,0.0,0.0,0.0,0.0,host,KKC103B
2,63.811922,0.644781,16.617554,0.0,0.0,4.147709,0.085269,1.756226,0.0,0.0,...,4.016627,2.876374,0.219198,0.0,0.0,0.0,0.0,0.0,host,KHD107B
3,68.591354,0.438112,15.640367,0.0,0.0,2.427771,0.063995,0.777234,0.0,0.0,...,4.550226,3.315973,0.162275,0.0,0.0,0.0,0.0,0.0,host,KCP109D
4,75.820463,0.081973,12.946199,0.0,0.0,0.663837,0.031223,0.089308,0.0,0.0,...,4.374870,4.009420,0.008794,0.0,0.0,0.0,0.0,0.0,aplite,KCP109C
5,67.622928,0.475611,15.900443,0.0,0.0,2.794764,0.056634,0.875509,0.0,0.0,...,4.469777,2.902462,0.195939,0.0,0.0,0.0,0.0,0.0,host,KCP114B
6,76.363613,0.069413,12.654557,0.0,0.0,0.517922,0.023925,0.054598,0.0,0.0,...,3.972797,4.609559,0.005901,0.0,0.0,0.0,0.0,0.0,aplite,KCP109B
7,65.146071,0.546541,16.125992,0.0,0.0,3.654955,0.072742,1.580363,0.0,0.0,...,3.658421,3.755211,0.195418,0.0,0.0,0.0,0.0,0.0,host,KHD106B
8,76.616213,0.079309,12.481429,0.0,0.0,0.623441,0.026245,0.055272,0.0,0.0,...,3.981179,4.460937,0.003877,0.0,0.0,0.0,0.0,0.0,aplite,KCP109A
9,69.494990,0.447260,14.881052,0.0,0.0,2.559606,0.061803,0.804506,0.0,0.0,...,4.268707,3.132743,0.168059,0.0,0.0,0.0,0.0,0.0,elizabeth,KCP108-M
